In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

len(columns)

86

In [5]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
ownership_num = {'MORTGAGE': 0, 'RENT': 1}
verification_num = {'Not Verified': 0, 'Verified': 1, 'Source Verified': 2}
issue_d_num = {'Mar-2019': 3, 'Feb-2019': 2, 'Jan-2019': 1}
risk_num = {'low_risk': 0, 'high_risk': 1}

clean_df = df.copy()

clean_df['home_ownership'] = clean_df['home_ownership'].apply(lambda x: 1 if x == 'RENT' else 0)

clean_df['loan_status'] = clean_df['loan_status'].apply(lambda x: 0 if x == 'low_risk' else 1)

clean_df['application_type'] = clean_df['application_type'].apply(lambda x: 0 if x == 'Joint App' else 1)

clean_df['verification_status'] = clean_df['verification_status'] \
                                    .apply(lambda x: {'Not Verified': 0, 'Verified': 1, 'Source Verified': 2}[x])

months = {'Jan-2019': 1, 'Feb-2019': 2, 'Mar-2019': 3, 'Apr-2019': 4, 'May-2019': 5, 'Jun-2019': 6,
          'Jul-2019': 7, 'Aug-2019': 8, 'Sep-2019': 9, 'Oct-2019': 10, 'Nov-2019': 11,
          'Dec-2019': 12}

for k, v in months.items():
    clean_df.replace(to_replace=k, value=v, inplace=True)

    
clean_df.replace(to_replace='n', value=0, inplace=True)
clean_df.replace(to_replace='y', value=1, inplace=True)
clean_df.replace(to_replace='N', value=0, inplace=True)
clean_df.replace(to_replace='Y', value=1, inplace=True)
clean_df.replace(to_replace='w', value=1, inplace=True)
clean_df.replace(to_replace='f', value=1, inplace=True)
clean_df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,1,66000.0,2,3,0,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,0,105000.0,1,3,0,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,0,56000.0,1,3,0,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,1,92000.0,1,3,0,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,0,52000.0,0,3,0,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,1,26000.0,2,1,0,0,9.60,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,0,0
68813,12000.0,0.2727,368.37,1,63000.0,0,1,0,0,29.07,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,0,0
68814,5000.0,0.1992,185.62,0,52000.0,2,1,0,0,14.86,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,0,0
68815,40000.0,0.0646,1225.24,0,520000.0,1,1,0,0,9.96,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,0,0


# Split the Data into Training and Testing

In [21]:
# Create our features
X =  clean_df[columns].copy()

# Create our target
y = df[target]

In [22]:
clean_df['pymnt_plan'].unique()

array([0], dtype=int64)

In [23]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,0.357659,8.821371e+04,0.895985,1.726172,0.005042,0.0,21.778153,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.479314,1.155800e+05,0.917165,0.743862,0.070831,0.0,20.199244,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,1.000000,0.000000,0.0,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,0.000000,5.000000e+04,0.000000,1.000000,0.000000,0.0,13.890000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,0.000000,7.300000e+04,1.000000,2.000000,0.000000,0.0,19.760000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,1.000000,1.040000e+05,2.000000,2.000000,0.000000,0.0,26.660000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,1.000000,8.797500e+06,2.000000,3.000000,1.000000,0.0,999.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [24]:
 X.dtypes.index[29]
 clean_df['application_type'].unique()
# list(X.dtypes)


array([1, 0], dtype=int64)

In [25]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [26]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_test)

Counter({'loan_status': 1})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [27]:
# Resample the training data with the BalancedRandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier

model = BalancedRandomForestClassifier(random_state=1)

In [28]:
# Calculated the balanced accuracy score

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

model = model.fit(X_train_scaled, y_train)
predictions = model.predict(X_test_scaled)
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.9981290926099158

In [29]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[  101,     0],
       [   64, 17040]], dtype=int64)

In [30]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.61      1.00      1.00      0.76      1.00      1.00       101
   low_risk       1.00      1.00      1.00      1.00      1.00      1.00     17104

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     17205



In [36]:
sorted(zip(X.columns, model.feature_importances_), reverse=True, key=lambda x: x[1])

[('loan_status', 0.39961407598387644),
 ('total_rec_prncp', 0.06502583116523508),
 ('total_pymnt', 0.045316869869077674),
 ('last_pymnt_amnt', 0.04360600562481706),
 ('total_pymnt_inv', 0.042963557689588656),
 ('total_rec_int', 0.036334435623993876),
 ('int_rate', 0.02011316345589393),
 ('issue_d', 0.0156713071288884),
 ('revol_bal', 0.010964160931142233),
 ('installment', 0.01053169206475867),
 ('avg_cur_bal', 0.010477272630890558),
 ('out_prncp', 0.009997181909655841),
 ('dti', 0.009698708140400462),
 ('bc_util', 0.009121990667518134),
 ('mths_since_recent_inq', 0.008925355647371874),
 ('out_prncp_inv', 0.0084935507259502),
 ('bc_open_to_buy', 0.008293053517528149),
 ('mths_since_rcnt_il', 0.0081347700844388),
 ('total_rev_hi_lim', 0.00792777350252308),
 ('max_bal_bc', 0.007837361243601897),
 ('tot_cur_bal', 0.007662798587550236),
 ('annual_inc', 0.007432889301132124),
 ('mo_sin_old_rev_tl_op', 0.007323312273511655),
 ('total_bal_il', 0.007246237272879625),
 ('mo_sin_rcnt_tl', 0.0067

In [37]:
# List the features sorted in descending order by feature importance
for k,v in sorted(zip(X.columns, model.feature_importances_), key=lambda x:x[1], reverse=True):
    print(f'{k}: ({v})')


loan_status: (0.39961407598387644)
total_rec_prncp: (0.06502583116523508)
total_pymnt: (0.045316869869077674)
last_pymnt_amnt: (0.04360600562481706)
total_pymnt_inv: (0.042963557689588656)
total_rec_int: (0.036334435623993876)
int_rate: (0.02011316345589393)
issue_d: (0.0156713071288884)
revol_bal: (0.010964160931142233)
installment: (0.01053169206475867)
avg_cur_bal: (0.010477272630890558)
out_prncp: (0.009997181909655841)
dti: (0.009698708140400462)
bc_util: (0.009121990667518134)
mths_since_recent_inq: (0.008925355647371874)
out_prncp_inv: (0.0084935507259502)
bc_open_to_buy: (0.008293053517528149)
mths_since_rcnt_il: (0.0081347700844388)
total_rev_hi_lim: (0.00792777350252308)
max_bal_bc: (0.007837361243601897)
tot_cur_bal: (0.007662798587550236)
annual_inc: (0.007432889301132124)
mo_sin_old_rev_tl_op: (0.007323312273511655)
total_bal_il: (0.007246237272879625)
mo_sin_rcnt_tl: (0.006767415979187566)
all_util: (0.006588838104924967)
inq_last_12m: (0.006553653713228177)
num_bc_tl: (0

### Easy Ensemble AdaBoost Classifier

In [38]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
model

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [40]:
# Calculated the balanced accuracy score
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

model = model.fit(X_train_scaled, y_train)
predictions = model.predict(X_test_scaled)
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.9981290926099158

In [41]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[  101,     0],
       [   64, 17040]], dtype=int64)

In [42]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.61      1.00      1.00      0.76      1.00      1.00       101
   low_risk       1.00      1.00      1.00      1.00      1.00      1.00     17104

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     17205

